In [ ]:
from voila.configuration import VoilaConfiguration
VoilaConfiguration.show_tracebacks = True

In [4]:
# ===========================
# CELL 1 - Upload + Time Window Selector (Plotly Version, Voilà-Safe)
# ===========================

import pandas as pd
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from io import StringIO

# --- Global variables to store data and widgets ---
df = None
fig = go.FigureWidget()
message_label = widgets.HTML()  # ← Voilà-safe status feedback

# --- Widgets ---
var_dropdown = widgets.Dropdown(description='Variable:')
time_slider = widgets.FloatRangeSlider(description='Time Window:', layout=widgets.Layout(width='95%'))
submit_button = widgets.Button(description="Trim to Window", button_style='danger')
upload = widgets.FileUpload(accept='.csv', multiple=False)
load_button = widgets.Button(description="Load Data", button_style='success')

# --- Setup controls ---
def setup_graph_controls():
    global fig
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    options = [col for col in numeric_cols if col != 'time']
    var_dropdown.options = options
    if options:
        var_dropdown.value = options[0]
    tmin, tmax = df['time_s'].min(), df['time_s'].max()
    time_slider.min = tmin
    time_slider.max = tmax
    time_slider.step = 0.01
    time_slider.value = (tmin, tmax)
    update_plot()

# --- Update plot ---
def update_plot(change=None):
    if df is None or var_dropdown.value is None:
        return
    var = var_dropdown.value
    t_start, t_end = time_slider.value
    mask = (df['time_s'] >= t_start) & (df['time_s'] <= t_end)
    fig.data = []
    fig.layout.shapes = []
    fig.add_trace(go.Scatter(x=df['time_s'][mask], y=df[var][mask], mode='lines', name=var))
    fig.update_layout(height=400, title=f"{var} vs Time")
    fig.add_shape(type='line', x0=t_start, x1=t_start, y0=0, y1=1, xref='x', yref='paper', line=dict(dash='dot'))
    fig.add_shape(type='line', x0=t_end, x1=t_end, y0=0, y1=1, xref='x', yref='paper', line=dict(dash='dot'))

# --- Trim to time window ---
def trim_data_to_window(b):
    global df
    t_start, t_end = time_slider.value
    df = df[(df['time_s'] >= t_start) & (df['time_s'] <= t_end)].reset_index(drop=True)
    message_label.value = f"<b>✅ Trimmed data to {t_start:.2f}s – {t_end:.2f}s.</b><br>New shape: {df.shape}"
    update_plot()

# --- Load CSV data ---
def load_data_from_upload(b):
    global df
    message_label.value = ""
    try:
        if not upload.value:
            message_label.value = "⚠️ Please upload a CSV file."
            return

        if isinstance(upload.value, dict):
            uploaded_file = list(upload.value.values())[0]
        elif isinstance(upload.value, (list, tuple)):
            uploaded_file = upload.value[0]
        else:
            uploaded_file = next(iter(upload.value.values()))

        content = uploaded_file['content']
        decoded = bytes(content).decode('utf-8', errors='replace')
        df = pd.read_csv(StringIO(decoded))

        if 'time' in df.columns:
            df['time_s'] = (df['time'] - df['time'].iloc[0]) / 1000.0
        else:
            df['time_s'] = df.index / 1000.0

        message_label.value = f"<b>✅ Loaded file with shape {df.shape}</b>"
        setup_graph_controls()

    except Exception as e:
        message_label.value = f"❌ Error loading data: {e}"

# --- Wire buttons and events ---
load_button.on_click(load_data_from_upload)
submit_button.on_click(trim_data_to_window)
var_dropdown.observe(update_plot, names='value')
time_slider.observe(update_plot, names='value')

# --- Display interface ---
display(widgets.VBox([
    widgets.HTML("<h3>Upload and Explore Data</h3>"),
    upload, load_button,
    var_dropdown,
    time_slider,
    fig,
    submit_button,
    message_label  # ← Voilà-friendly feedback
]))

In [ ]:
# ===========================
# CELL 2 - Sensor Offset Manager (Run Step + Axis Flip Toggle)
# ===========================

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt

# --- Globals ---
df_raw = None
offset_values = {
    'accel': {'x': 0.0, 'y': 0.0, 'z': 0.0},
    'gyro':  {'x': 0.0, 'y': 0.0, 'z': 0.0},
    'mag':   {'x': 0.0, 'y': 0.0, 'z': 0.0}
}
parsed_groups = []

# --- Widgets ---
step2_button = widgets.Button(description="▶ Run Sensor Offset Manager", button_style='primary')
step2_output = widgets.Output()
mode_selector = widgets.RadioButtons(options=['Original Data', 'Flipped Data'], description='View:', style={'description_width': 'initial'})

sensor_selector = widgets.Dropdown(options=['accel', 'gyro', 'mag'], description="Sensor:")
method_selector = widgets.Dropdown(options=['Manual Entry', 'From CALIB.TXT'], description="Method:")

x_offset = widgets.FloatText(description="X")
y_offset = widgets.FloatText(description="Y")
z_offset = widgets.FloatText(description="Z")

upload_txt = widgets.FileUpload(accept=".txt", multiple=False)
parse_button = widgets.Button(description="📂 Parse CALIB.TXT", button_style='success')
group_selector = widgets.Dropdown(options=[], description="Select Group:")
apply_button = widgets.Button(description="Apply Offsets", button_style="primary")
reset_button = widgets.Button(description="Reset Offsets", button_style="danger")

offset_output = widgets.Output()
offset_table = widgets.Output()
plot_output = widgets.Output()
container = widgets.VBox()

# --- Functions ---
def update_offset_table():
    with offset_table:
        clear_output()
        df_disp = pd.DataFrame({
            'Accelerometer': [offset_values['accel']['x'], offset_values['accel']['y'], offset_values['accel']['z']],
            'Gyroscope':     [offset_values['gyro']['x'], offset_values['gyro']['y'], offset_values['gyro']['z']],
            'Magnetometer':  [offset_values['mag']['x'],  offset_values['mag']['y'],  offset_values['mag']['z']]
        }, index=['X', 'Y', 'Z'])
        display(df_disp.round(4))

def update_method_ui(*args):
    method = method_selector.value
    container.children = [widgets.HBox([sensor_selector, method_selector])]

    if method == "Manual Entry":
        container.children += (widgets.HBox([x_offset, y_offset, z_offset]),)
    elif method == "From CALIB.TXT":
        container.children += (upload_txt, parse_button, group_selector)

    container.children += (widgets.HBox([apply_button, reset_button]), offset_table, offset_output, plot_output)

def parse_calib_txt(b):
    global parsed_groups
    with offset_output:
        clear_output()
        if not upload_txt.value:
            print("Please upload a CALIB.TXT file.")
            return
        uploaded = upload_txt.value[0] if isinstance(upload_txt.value, (list, tuple)) else list(upload_txt.value.values())[0]
        content = bytes(uploaded['content']).decode('utf-8', errors='replace')
        lines = [line.strip() for line in content.strip().splitlines() if line.strip()]
        groups = []
        i = 0
        while i < len(lines):
            if lines[i].lower().startswith("accel bias"):
                try:
                    accel = list(map(float, lines[i+1].split(',')))
                    gyro  = list(map(float, lines[i+3].split(',')))
                    mag   = list(map(float, lines[i+5].split(',')))
                    groups.append({'accel': accel, 'gyro': gyro, 'mag': mag})
                    i += 6
                except Exception as e:
                    print(f"⚠️ Skipped group due to parsing error: {e}")
                    i += 1
            else:
                i += 1
        if not groups:
            print("No valid calibration groups found.")
            return
        parsed_groups = groups
        group_selector.options = [f"Group {i}" for i in range(len(groups))]
        print(f"✅ Parsed {len(groups)} groups. Select one to apply.")
parse_button.on_click(parse_calib_txt)

def apply_offsets(b):
    global df_raw, df
    with offset_output:
        clear_output()
        if df is None:
            print("⚠️ Load data in Step 1 first.")
            return
        if df_raw is None:
            df_raw = df.copy()

        sensor = sensor_selector.value
        method = method_selector.value

        if method == "Manual Entry":
            offset_values[sensor]['x'] = x_offset.value
            offset_values[sensor]['y'] = y_offset.value
            offset_values[sensor]['z'] = z_offset.value
        elif method == "From CALIB.TXT":
            if not parsed_groups:
                print("⚠️ No calibration file parsed.")
                return
            group = parsed_groups[group_selector.index]
            bias = group[sensor]
            offset_values[sensor] = dict(zip(['x','y','z'], bias))

        df = df_raw.copy()
        for sens in ['accel', 'gyro', 'mag']:
            for axis in ['x', 'y', 'z']:
                col = f"{sens[0]}{axis}"
                if col in df.columns:
                    df[col] -= offset_values[sens][axis]

        print(f"✅ Applied offsets to {sensor.upper()} (subtracted):", offset_values[sensor])
        update_offset_table()
    update_plot()

def reset_offsets(b):
    global df_raw, df
    if df_raw is not None:
        df = df_raw.copy()
    for sens in offset_values:
        for axis in offset_values[sens]:
            offset_values[sens][axis] = 0.0
    update_offset_table()
    with offset_output:
        clear_output()
        print("🔁 All offsets reset.")
    with plot_output:
        clear_output()

def update_plot():
    with plot_output:
        clear_output()
        if df is None:
            return
        df_plot = df.copy()
        if mode_selector.value == "Flipped Data":
            if all(axis in df_plot.columns for axis in ['ax', 'ay', 'az']):
                ax, ay, az = df_plot['ax'], df_plot['ay'], df_plot['az']
                df_plot['ax'] = -az
                df_plot['ay'] = ay
                df_plot['az'] = ax
            if all(axis in df_plot.columns for axis in ['gx', 'gy', 'gz']):
                gx, gy, gz = df_plot['gx'], df_plot['gy'], df_plot['gz']
                df_plot['gx'] = -gz
                df_plot['gy'] = gy
                df_plot['gz'] = gx
            if all(axis in df_plot.columns for axis in ['mx', 'my', 'mz']):
                mx, my, mz = df_plot['mx'], df_plot['my'], df_plot['mz']
                df_plot['mx'] = mx
                df_plot['my'] = mz
                df_plot['mz'] = my

        sensor = sensor_selector.value
        sensor_prefix = {'accel': 'a', 'gyro': 'g', 'mag': 'm'}[sensor]
        labels = [f"{sensor_prefix}x", f"{sensor_prefix}y", f"{sensor_prefix}z"]
        titles = {'accel': "Accelerometer", 'gyro': "Gyroscope", 'mag': "Magnetometer"}
        units = {'accel': "(g)", 'gyro': "(°/s)", 'mag': "(µT)"}

        fig, ax = plt.subplots(figsize=(12, 4))
        for label in labels:
            if label in df_plot.columns:
                ax.plot(df_plot['time_s'], df_plot[label], label=label)
        ax.set_title(f"{titles[sensor]} - {mode_selector.value}")
        ax.set_ylabel(units[sensor])
        ax.set_xlabel("Time (s)")
        ax.legend()
        plt.tight_layout()
        plt.show()

def initialize_mag_offsets():
    if df is None:
        return
    for axis in ['x', 'y', 'z']:
        col = f"m{axis}"
        if col in df.columns:
            offset_values['mag'][axis] = (df[col].max() + df[col].min()) / 2

def run_step2(_):
    with step2_output:
        clear_output()
        initialize_mag_offsets()
        update_offset_table()
        update_plot()
        print("✅ Step 2 executed. Apply offsets or view flipped data.")
    update_method_ui()

# --- Wiring ---
apply_button.on_click(apply_offsets)
reset_button.on_click(reset_offsets)
sensor_selector.observe(update_method_ui, names='value')
method_selector.observe(update_method_ui, names='value')
mode_selector.observe(lambda _: update_plot(), names='value')
step2_button.on_click(run_step2)

# --- Display ---
display(widgets.HTML("<h3>Step 2: 🛠️ Sensor Offset Manager</h3>"))
display(step2_button, mode_selector, step2_output, container)

In [ ]:
# ===========================
# CELL 3 - Preprocess + Sliders for Event Marking
# ===========================

import numpy as np
import pandas as pd
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Step button to trigger all logic ---
run_step3_button = widgets.Button(description="▶️ Run Basic Event Marking", button_style='primary')
step3_output = widgets.Output()

def run_step3(b):
    with step3_output:
        clear_output()

        # --- Constants for Hypsometric formula ---
        P0 = 1013.25
        T0 = 288.15
        g = 9.80665
        R = 287.05

        global df, marker_data
        df = df.copy()
        df['time_s'] = (df['time'] - df['time'].iloc[0]) / 1000.0
        df['temp_K'] = df['temp'] + 273.15 if df['temp'].max() < 100 else df['temp']

        buffer = 0.75
        launch_guess = df['time_s'].iloc[50]
        pre_launch_mask = df['time_s'] < (launch_guess - buffer)
        avg_prelaunch_pressure = df.loc[pre_launch_mask, 'pressure'].mean()
        initial_elevation_m = (R * T0 / g) * np.log(P0 / avg_prelaunch_pressure)

        df['altitude_m'] = (R * T0 / g) * np.log(P0 / df['pressure'])
        df['altitude_m'] -= df['altitude_m'].iloc[0]

        marker_data = {
            'launch': {'time': df['time_s'].iloc[50]},
            'apogee': {'time': df['time_s'].iloc[len(df)//2]},
            'landing': {'time': df['time_s'].iloc[-50]}
        }

        fig = go.FigureWidget()
        fig.add_scatter(x=df['time_s'], y=df['altitude_m'], mode='lines', name='Altitude')

        colors = {'launch': 'green', 'apogee': 'orange', 'landing': 'red'}
        shape_template = lambda name, t: dict(type='line', xref='x', yref='paper', x0=t, x1=t, y0=0, y1=1,
                                              line=dict(color=colors[name], dash='dot'))

        fig.update_layout(height=500, width=700, title_text="Altitude Over Time")
        fig.update_layout(shapes=[shape_template(k, marker_data[k]['time']) for k in marker_data])

        sliders = {
            k: widgets.FloatSlider(value=marker_data[k]['time'], min=df['time_s'].min(), max=df['time_s'].max(), step=0.01,
                                    description=f"{k.capitalize()} (s)", continuous_update=True,
                                    layout=widgets.Layout(width='95%'))
            for k in marker_data
        }
        labels = {k: widgets.Label() for k in marker_data}

        def update_lines(change=None):
            for i, k in enumerate(marker_data):
                t = sliders[k].value
                marker_data[k]['time'] = t
                fig.layout.shapes[i].x0 = t
                fig.layout.shapes[i].x1 = t
                alt = df.iloc[(df['time_s'] - t).abs().argsort()[:1]]['altitude_m'].values[0]
                labels[k].value = f"{k.capitalize()} @ {t:.2f} s → Alt: {alt:.2f} m"
            fig.layout.annotations = []
            for k in marker_data:
                t = marker_data[k]['time']
                alt = df.iloc[(df['time_s'] - t).abs().argsort()[:1]]['altitude_m'].values[0]
                fig.add_annotation(x=t, y=alt, text=k.capitalize(), showarrow=True, arrowhead=1)

        for slider in sliders.values():
            slider.observe(update_lines, names='value')

        submit_output = widgets.Output()
        def submit_markers(b):
            with submit_output:
                submit_output.clear_output()
                print("✅ Markers submitted!")
                for k in marker_data:
                    print(f"  {k.capitalize()}: {marker_data[k]['time']:.2f} s")

        submit_button = widgets.Button(description="✅ Submit Markers", button_style='success')
        submit_button.on_click(submit_markers)

        update_lines()

        display(widgets.VBox([
            widgets.HTML("<h4>📍 Use Sliders to Mark Launch, Apogee, and Landing</h4>"),
            *[widgets.VBox([sliders[k], labels[k]]) for k in marker_data],
            fig,
            submit_button,
            submit_output
        ]))

run_step3_button.on_click(run_step3)
display(widgets.VBox([run_step3_button, step3_output]))


In [ ]:
# ===========================
# CELL 4A -- Map Marker
# ===========================

import ipywidgets as widgets
from ipyleaflet import Map, Marker, AwesomeIcon, basemap_to_tiles, basemaps
from IPython.display import display, clear_output

# Output widget
map_out = widgets.Output()

# State
map_points = {'launch': None, 'apogee': None, 'landing': None}
saved_map_points = {}
markers = []

# Inputs & buttons
lat_input = widgets.FloatText(description='Latitude', value=39.5951)
lon_input = widgets.FloatText(description='Longitude', value=-77.6142)
set_location_btn = widgets.Button(description="Set Launch Site", button_style='info')
save_btn = widgets.Button(description="✅ Save Points", button_style='success')
reset_btn = widgets.Button(description="🔄 Reset", button_style='warning')
run_btn = widgets.Button(description="▶ Map Marker", button_style='primary')

# Handlers
def handle_click(**kwargs):
    if kwargs.get('type') != 'click':
        return
    lat, lon = kwargs['coordinates']
    label, icon = None, None
    if map_points['launch'] is None:
        label, icon = 'launch', AwesomeIcon(name='rocket', marker_color='green')
    elif map_points['apogee'] is None:
        label, icon = 'apogee', AwesomeIcon(name='arrow-up', marker_color='orange')
    elif map_points['landing'] is None:
        label, icon = 'landing', AwesomeIcon(name='flag-checkered', marker_color='red')
    else:
        return

    marker = Marker(location=(lat, lon), icon=icon, draggable=True)
    marker.observe(make_drag_handler(label), names='location')
    m.add_layer(marker)
    markers.append(marker)
    map_points[label] = (lat, lon)

    with map_out:
        print(f"✅ {label.capitalize()} set at: ({lat:.5f}, {lon:.5f})")

def make_drag_handler(label):
    def on_drag(change):
        map_points[label] = tuple(change['new'])
        with map_out:
            print(f"🔄 {label.capitalize()} moved to: {map_points[label]}")
    return on_drag

def on_set_location(b):
    global m
    with map_out:
        clear_output()
        m = Map(center=(lat_input.value, lon_input.value), zoom=18,
                basemap=basemap_to_tiles(basemaps.Esri.WorldImagery))
        m.on_interaction(handle_click)
        display(widgets.VBox([m, widgets.HBox([save_btn, reset_btn])]))

def on_save(b):
    global saved_map_points
    saved_map_points = map_points.copy()
    with map_out:
        print("✅ Points saved for Step 4B.")

def on_reset(b):
    for mk in markers:
        m.remove_layer(mk)
    markers.clear()
    for k in map_points:
        map_points[k] = None
    with map_out:
        clear_output()
        print("🔄 Reset complete.")

# Run-step handler
def run_step_4a(b):
    with map_out:
        clear_output()
        display(widgets.VBox([
            lat_input, lon_input,
            set_location_btn
        ]))

# Wire buttons
run_btn.on_click(run_step_4a)
set_location_btn.on_click(on_set_location)
save_btn.on_click(on_save)
reset_btn.on_click(on_reset)

# Display UI
display(run_btn, map_out)

In [ ]:
# ===========================
# UPDATED CELL 4B - Enhanced Displacement Plot with Saved Metrics
# ===========================

from ipywidgets import Button, Output, VBox
from IPython.display import display, clear_output
from pyproj import Geod
import matplotlib.pyplot as plt
import numpy as np

# --- Outputs & Button ---
displacement_output = Output()
run_4b_button = Button(description="▶ Run Displacement Plot", button_style='primary')
geod = Geod(ellps='WGS84')

# --- Callback ---
def run_step_4b(b):
    global displacement_data
    displacement_data = {}

    with displacement_output:
        clear_output()

        if not map_points.get('launch') or not map_points.get('landing'):
            print("⚠️ Please set both launch and landing points in Step 4A.")
            return

        lat1, lon1 = map_points['launch']
        lat3, lon3 = map_points['landing']
        az_l, _, dist_l = geod.inv(lon1, lat1, lon3, lat3)
        dx_l = dist_l * np.cos(np.radians(az_l))
        dy_l = dist_l * np.sin(np.radians(az_l))

        # Save launch→landing data
        displacement_data.update({
            'azimuth_launch_to_landing': az_l,
            'distance': dist_l,
            'dx': dx_l,
            'dy': dy_l,
        })

        print(f"📍 Launch → Landing Azimuth: {az_l:.2f}°")
        print(f"📏 Distance: {dist_l:.2f} m | ΔY: {dy_l:.2f} m | ΔX: {dx_l:.2f} m")

        plt.figure(figsize=(7, 7))

        if map_points.get('apogee'):
            lat2, lon2 = map_points['apogee']
            az_a, _, dist_a = geod.inv(lon1, lat1, lon2, lat2)
            dx_a = dist_a * np.cos(np.radians(az_a))
            dy_a = dist_a * np.sin(np.radians(az_a))

            # Save launch→apogee data
            displacement_data.update({
                'azimuth_launch_to_apogee': az_a,
                'distance_apogee': dist_a,
                'dx_apogee': dx_a,
                'dy_apogee': dy_a,
            })

            print(f"\n📍 Launch → Apogee Azimuth: {az_a:.2f}°")
            print(f"📏 Distance: {dist_a:.2f} m | ΔY: {dy_a:.2f} m | ΔX: {dx_a:.2f} m")

            plt.plot([0, dy_a, dy_l], [0, dx_a, dx_l], 'bo-', label='Flight Path')
            plt.text(0, 0, 'Launch', ha='right', va='top')
            plt.text(dy_a, dx_a, 'Apogee', ha='center', va='bottom')
            plt.text(dy_l, dx_l, 'Landing', ha='left', va='top')

            # Launch → Apogee helpers
            plt.plot([0, dy_a], [0, 0], 'r:', alpha=0.7)
            plt.plot([dy_a, dy_a], [0, dx_a], 'g:', alpha=0.7)
            plt.annotate(f"{dy_a:.1f} m", xy=(dy_a/2, -2), ha='center', color='red')
            plt.annotate(f"{dx_a:.1f} m", xy=(dy_a + 1, dx_a/2), va='center', color='green')

        else:
            plt.plot([0, dy_l], [0, dx_l], 'bo-', label='Launch → Landing')
            plt.text(0, 0, "Launch", ha='right', va='top')
            plt.text(dy_l, dx_l, "Landing", ha='left', va='bottom')

        # Launch → Landing helpers
        plt.plot([0, dy_l], [0, 0], 'r:', alpha=0.7)
        plt.plot([dy_l, dy_l], [0, dx_l], 'g:', alpha=0.7)
        plt.annotate(f"{dy_l:.1f} m", xy=(dy_l/2, -4), ha='center', color='red')
        plt.annotate(f"{dx_l:.1f} m", xy=(dy_l + 1, dx_l/2), va='center', color='green')

        plt.grid(True)
        plt.axis('equal')
        plt.xlabel("ΔY (meters, East)")
        plt.ylabel("ΔX (meters, North)")
        plt.title("XY Displacement from Launch → Apogee → Landing")
        plt.legend()
        plt.tight_layout()
        plt.show()

run_4b_button.on_click(run_step_4b)
display(VBox([run_4b_button, displacement_output]))

In [ ]:
# ===========================
# CELL 4C - Environment Input + Magnetic Field Estimation (NOAA-style)
# ===========================

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

# --- Ensure shared state exists ---
if 'displacement_data' not in globals():
    displacement_data = {}

# --- Output and button ---
env_output = widgets.Output()
run_4c_button = widgets.Button(description="▶ Save Environment Data", button_style='primary')

# --- Inputs ---
temp_input = widgets.FloatText(description='Temperature (°C)', value=20.0)
wind_speed_input = widgets.FloatText(description='Wind Speed (kph)', value=5.0)
wind_dir_input = widgets.Text(description='Wind Direction', value='N')

# --- Simplified magnetic field estimation ---
def estimate_mag_field(lat, lon):
    declination = 10.0
    inclination = 60.0
    strength = 50_000
    H = strength * np.cos(np.radians(inclination))
    X = H * np.cos(np.radians(declination))
    Y = H * np.sin(np.radians(declination))
    Z = -strength * np.sin(np.radians(inclination))
    return {'X_uT': X / 1000, 'Y_uT': Y / 1000, 'Z_uT': Z / 1000}

# --- Callback ---
def run_step_4c(b):
    global displacement_data
    with env_output:
        clear_output()
        if 'map_points' not in globals() or not map_points.get('launch'):
            print("⚠️ Please set the launch point first in Step 4A.")
            return

        lat, lon = map_points['launch']
        temp = temp_input.value
        wind_kph = wind_speed_input.value
        wind_dir = wind_dir_input.value.strip()

        mag = estimate_mag_field(lat, lon)

        print(f"🌡️ Temperature: {temp:.1f} °C")
        print(f"💨 Wind: {wind_kph:.1f} kph from {wind_dir}")
        print("🧭 Estimated Magnetic Field (μT):")
        for k, v in mag.items():
            print(f"  {k}: {v:.2f}")

        displacement_data.update({
            'temperature_C': temp,
            'wind_kph': wind_kph,
            'wind_dir': wind_dir,
            **mag
        })

        print("✅ Environmental and magnetic data saved to `displacement_data`.")

run_4c_button.on_click(run_step_4c)

# --- UI display ---
display(widgets.VBox([
    run_4c_button,
    temp_input,
    wind_speed_input,
    wind_dir_input,
    env_output
]))

In [ ]:
# ===========================
# CELL 5 - Improved Motion Profile with Still-Zeroing and Light Smoothing
# ===========================

# --- Imports ---
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

run5_output = widgets.Output()
run5_button = widgets.Button(description="▶ Run Improved Motion Profile", button_style='primary')

def run_step_5(b):
    with run5_output:
        clear_output()

        # --- Time Segments and Flight Window ---
        dt = df['time_s'].diff().fillna(0)
        launch = marker_data['launch']['time']
        landing = marker_data['landing']['time']
        flight_mask = (df['time_s'] >= launch) & (df['time_s'] <= landing)

        # --- Compute Velocity ---
        raw_vel = df['altitude_m'].diff() / dt
        vel_smoothed = raw_vel.rolling(window=5, center=True, min_periods=1).mean()
        vel = np.zeros_like(df['altitude_m'])
        vel[flight_mask] = vel_smoothed[flight_mask]
        df['vel_mps'] = vel

        # --- Compute Acceleration ---
        raw_accel = df['vel_mps'].diff() / dt
        accel_smoothed = raw_accel.rolling(window=5, center=True, min_periods=1).mean()
        accel = np.zeros_like(df['altitude_m'])
        accel[flight_mask] = accel_smoothed[flight_mask]
        df['accel_mps2'] = accel
        df['net_accel_mps2'] = df['accel_mps2']

        # --- Marker Time Defaults ---
        def get_marker_time(name, fallback):
            try:
                return marker_data[name]['time']
            except:
                return fallback

        t_min, t_max = df['time_s'].min(), df['time_s'].max()
        defaults = {
            'launch': get_marker_time('launch', df['time_s'].iloc[50]),
            'apogee': get_marker_time('apogee', df['time_s'].iloc[len(df)//2]),
            'landing': get_marker_time('landing', df['time_s'].iloc[-50]),
        }
        defaults['burnout'] = defaults['launch'] + 1.5
        defaults['parachute'] = defaults['apogee'] + 1.0

        colors = {
            'launch': 'green', 'burnout': 'purple', 'apogee': 'orange',
            'parachute': 'brown', 'landing': 'red'
        }
        ordered_keys = ['launch', 'burnout', 'apogee', 'parachute', 'landing']
        sliders = {
            name: widgets.FloatSlider(
                value=defaults[name],
                min=t_min,
                max=t_max,
                step=0.01,
                description=name.capitalize(),
                continuous_update=True,
                layout=widgets.Layout(width='95%')
            )
            for name in ordered_keys
        }

        # --- Plot ---
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                            subplot_titles=("Altitude (m)", "Velocity (m/s)", "Acceleration (m/s²)"))
        fig.add_trace(go.Scatter(x=df['time_s'], y=df['altitude_m'], name='Altitude'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df['time_s'], y=df['vel_mps'], name='Velocity'), row=2, col=1)
        fig.add_trace(go.Scatter(x=df['time_s'], y=df['net_accel_mps2'], name='Net Accel'), row=3, col=1)

        fig.update_layout(
            height=700, width=700,
            title_text="Flight Profile Using Temp/Pressure",
            shapes=[
                dict(type='line', xref='x', yref='paper', x0=sliders[k].value, x1=sliders[k].value,
                     y0=0, y1=1, line=dict(color=colors[k], dash='dot'))
                for k in ordered_keys
            ]
        )
        fig.update_yaxes(showgrid=True)
        figw = go.FigureWidget(fig)

        # --- Update Plot ---
        def update_plot(change=None):
            for i, name in enumerate(ordered_keys):
                val = sliders[name].value
                figw.layout.shapes[i].x0 = val
                figw.layout.shapes[i].x1 = val

        for slider in sliders.values():
            slider.observe(update_plot, names='value')

        # --- Save Button ---
        output = widgets.Output()
        def store_final_data(b):
            for k in ordered_keys:
                marker_data[k] = {'time': sliders[k].value}
            global filtered_data_by_var
            if 'filtered_data_by_var' not in globals():
                filtered_data_by_var = {}
            filtered_data_by_var['vel_mps'] = df['vel_mps'].copy()
            filtered_data_by_var['accel_mps2'] = df['accel_mps2'].copy()
            with output:
                output.clear_output()
                print("✅ All data saved!")
                print("📊 Flight Summary:")
                print(f"  Max Altitude: {df['altitude_m'].max():.2f} m")
                print(f"  Max Velocity: {df['vel_mps'].max():.2f} m/s")
                print(f"  Max Acceleration: {df['net_accel_mps2'].max():.2f} m/s²")
                print(f"  Total Flight Time: {sliders['landing'].value - sliders['launch'].value:.2f} s")
                print(f"  Ascent Time: {sliders['apogee'].value - sliders['launch'].value:.2f} s")
                print(f"  Descent Time: {sliders['landing'].value - sliders['apogee'].value:.2f} s")
                print(f"  Burnout: {sliders['burnout'].value:.2f} s")
                print(f"  Parachute: {sliders['parachute'].value:.2f} s")

        save_all_button = widgets.Button(description="✅ Save All", button_style='success')
        save_all_button.on_click(store_final_data)

        display(widgets.VBox([
            widgets.HTML("<h4>🎛️ Adjust Events Using Sliders (Launch → Landing)</h4>"),
            *[sliders[name] for name in ordered_keys],
            figw,
            save_all_button,
            output
        ]))

run5_button.on_click(run_step_5)

# --- Display Trigger ---
display(run5_button, run5_output)

In [ ]:
# ===========================
# CELL 6 - Sensor Noise Summary with Targeted Highlighting
# ===========================

# --- Imports ---
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import ipywidgets as widgets

# --- UI Setup ---
step6_button = widgets.Button(description="▶ View Sensor Noise Summary", button_style='primary')
step6_output = widgets.Output()

def run_step_6(b):
    with step6_output:
        clear_output()

        buffer = 0.75
        launch_time = marker_data['launch']['time']
        landing_time = marker_data['landing']['time']
        pre_df = df[df['time_s'] < (launch_time - buffer)]
        post_df = df[df['time_s'] > (landing_time + buffer)]

        imu_vars = ['ax', 'ay', 'az', 'gx', 'gy', 'gz', 'mx', 'my', 'mz']
        env_vars = ['temp', 'pressure']
        sensor_vars = [var for var in imu_vars + env_vars if var in df.columns]

        summary_data = []
        for var in sensor_vars:
            pre = pre_df[var].dropna()
            post = post_df[var].dropna()
            pre_mean = pre.mean()
            post_mean = post.mean()
            pre_std = pre.std()
            post_std = post.std()
            pre_cv = pre_std / (abs(pre_mean) + 1e-6)
            post_cv = post_std / (abs(post_mean) + 1e-6)
            summary_data.append({
                'Variable': var,
                'Pre Mean': pre_mean,
                'Pre Std': pre_std,
                'Pre CV': pre_cv,
                'Post Mean': post_mean,
                'Post Std': post_std,
                'Post CV': post_cv
            })

        noise_summary = pd.DataFrame(summary_data).set_index('Variable')

        def highlight_cells(val, col, var):
            if 'CV' in col and val > 0.2:
                return 'background-color: salmon'
            if 'Mean' in col and var in ['ax', 'ay', 'az'] and not (-1.0 <= val <= 1.0):
                return 'background-color: lightblue'
            return ''

        def highlight_df(df):
            return df.style.apply(
                lambda row: [highlight_cells(row[col], col, row.name) for col in df.columns],
                axis=1
            ).format("{:.4f}").set_caption("Sensor Noise Summary (Pre/Post)")

        styled_table = highlight_df(noise_summary)

        # --- Bar Plot ---
        cv_df = noise_summary[['Pre CV', 'Post CV']].iloc[::-1]
        fig, ax = plt.subplots(figsize=(6, len(cv_df) * 0.4))
        x = np.arange(len(cv_df))
        width = 0.35
        ax.barh(x - width/2, cv_df['Pre CV'], height=width, label='Preflight CV')
        ax.barh(x + width/2, cv_df['Post CV'], height=width, label='Postflight CV')
        ax.set_yticks(x)
        ax.set_yticklabels(cv_df.index)
        ax.set_xlabel("Coefficient of Variation (CV)")
        ax.set_title("Sensor Noise (Pre vs Post)")
        ax.legend()
        ax.grid(True, axis='x', linestyle='--', alpha=0.6)
        plt.tight_layout()

        out1 = widgets.Output()
        out2 = widgets.Output()
        with out1:
            display(styled_table)
        with out2:
            plt.show()

        display(widgets.HBox([out1, out2]))

        # --- Save for downstream use ---
        global sensor_noise_summary
        sensor_noise_summary = noise_summary.copy()

step6_button.on_click(run_step_6)

# --- Display entry point ---
display(step6_button, step6_output)

In [ ]:
# ===========================
# CELL 7 - Filtering with Mode Selector (Plotly Version)
# ===========================

# --- Imports ---
import numpy as np
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objs as go
from IPython.display import display, clear_output

# --- Button and Output ---
run_button_7 = widgets.Button(description="▶ Run Filter Selection", button_style='primary')
run_output_7 = widgets.Output()

def run_step_7(b):
    with run_output_7:
        clear_output()

        global filtered_data_by_var, filter_params
        filtered_data_by_var = {}
        filter_params = {}

        buffer = 0.75
        launch = marker_data['launch']['time']
        landing = marker_data['landing']['time']
        t_min, t_max = df['time_s'].min(), df['time_s'].max()
        pre_df = df[df['time_s'] < (launch - buffer)]
        post_df = df[df['time_s'] > (landing + buffer)]

        for axis in ['x', 'y', 'z']:
            col = f'g{axis}'
            if col in df.columns:
                pre_bias = pre_df[col].mean()
                post_bias = post_df[col].mean()
                t = df['time_s']
                flight_mask = (t >= launch) & (t <= landing)
                blend = np.clip((t - launch) / (landing - launch), 0, 1)
                bias_interp = (1 - blend) * pre_bias + blend * post_bias
                df[f'{col}_corr'] = df[col].copy()
                df.loc[flight_mask, f'{col}_corr'] -= bias_interp[flight_mask]
                df.loc[t < launch, f'{col}_corr'] -= pre_bias
                df.loc[t > landing, f'{col}_corr'] -= post_bias

        filter_vars = ['ax', 'ay', 'az', 'gx_corr', 'gy_corr', 'gz_corr', 'mx', 'my', 'mz', 'temp', 'pressure']
        filter_vars = [var for var in filter_vars if var in df.columns]
        cv_lookup = sensor_noise_summary[['Pre CV', 'Post CV']].mean(axis=1).to_dict()

        def apply_filter(intensity_scale=1.0):
            event_times = [launch, marker_data['burnout']['time'], marker_data['apogee']['time'],
                   marker_data['parachute']['time'], landing]
            def filter_strength(time, event_width=0.75, min_strength=0.25):
                strength = np.ones_like(time)
                for event_time in event_times:
                    dist = np.abs(time - event_time)
                    strength *= np.clip(dist / event_width, min_strength, 1.0)
                return strength
        
            for var in filter_vars:
                base_cv = cv_lookup.get(var.replace('_corr', ''), 0.02)
                base_win = int(min(51, max(5, int(15 + 400 * base_cv))))
                weights = filter_strength(df['time_s'].values)
                dynamic_window = np.clip((base_win * weights * intensity_scale).astype(int), a_min=3, a_max=None)
                smoothed = []
                for i in range(len(df)):
                    half_win = dynamic_window[i] // 2
                    lo = max(0, i - half_win)
                    hi = min(len(df), i + half_win + 1)
                    smoothed.append(df[var].iloc[lo:hi].mean())
                df[f'{var}_filtered'] = smoothed

        mode_dropdown = widgets.Dropdown(
            options=['Suggested Filter', 'Custom', 'No Filter'],
            value='Suggested Filter',
            description='Mode:'
        )
        intensity_slider = widgets.IntSlider(
            value=50, min=0, max=100, step=1,
            description='Filter Strength:',
            layout=widgets.Layout(width='95%')
        )
        dropdown = widgets.Dropdown(
            options=filter_vars,
            value='az' if 'az' in filter_vars else filter_vars[0],
            description='Variable:',
            layout=widgets.Layout(width='300px')
        )
        output = widgets.Output()
        save_output = widgets.Output()

        def update_all(change=None):
            mode = mode_dropdown.value
            if mode == 'No Filter':
                for var in filter_vars:
                    df[f'{var}_filtered'] = df[var]
                intensity_slider.disabled = True
            else:
                intensity_slider.disabled = (mode != 'Custom')
                scale = 0.01 + (intensity_slider.value / 100) ** 2 * 2.0
                apply_filter(scale)

            var = dropdown.value
            raw = df[var]
            filtered = df[f'{var}_filtered']
            t = df['time_s']
            output.clear_output(wait=True)
            with output:
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=t, y=raw, name='Raw', line=dict(color='blue', width=1)))
                fig.add_trace(go.Scatter(x=t, y=filtered, name='Filtered', line=dict(color='red', width=2)))
                fig.update_layout(title=f"{var} - Raw vs Filtered (Mode: {mode})", xaxis_title="Time (s)", yaxis_title="Sensor Value",
                                  height=400, legend=dict(x=0.01, y=0.99), margin=dict(l=40, r=20, t=40, b=40))
                fig.show()

        save_button = widgets.Button(description="✅ Save Filtered Data", button_style='success')
        def save_filtered_data(b):
            with save_output:
                save_output.clear_output()
                scale = 0.01 + (intensity_slider.value / 100) ** 2 * 2.0
                for var in filter_vars:
                    filtered_data_by_var[var] = df[f'{var}_filtered'].copy()
                    filter_params[var] = {'intensity_slider': intensity_slider.value, 'scale': scale}
                print("✅ Filtered data and parameters saved!")

        save_button.on_click(save_filtered_data)
        mode_dropdown.observe(update_all, names='value')
        intensity_slider.observe(update_all, names='value')
        dropdown.observe(update_all, names='value')

        display(widgets.VBox([
            widgets.HTML("<h4>🧹 Filter Signals (No Filter, Suggested, or Custom)</h4>"),
            mode_dropdown,
            intensity_slider,
            dropdown,
            output,
            save_button,
            save_output
        ]))
        update_all()

# --- Hook up the button and output ---
run_button_7.on_click(run_step_7)
container_7 = widgets.VBox([run_button_7, run_output_7])
display(container_7)

In [ ]:
# ===========================
# CELL 8 - Orientation Estimation and Transformation to World Frame
# ===========================

# --- Imports ---
import numpy as np
from ahrs.filters import Madgwick
from scipy.spatial.transform import Rotation as R
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Entry Point ---
run_button_8 = widgets.Button(description="▶ View World Frame Orientation", button_style='primary')
run_output_8 = widgets.Output()

def run_step_8(b):
    with run_output_8:
        clear_output()

        # Step 1: Remove Preflight Accelerometer Bias
        preflight = df[df['time_s'] < marker_data['launch']['time'] - 0.5]
        acc_bias = preflight[['ax', 'ay', 'az']].mean().values
        df[['ax', 'ay', 'az']] -= acc_bias

        # Step 2: Extract and Convert Sensor Data
        acc = df[['ax','ay','az']].values
        gyro = df[['gx_corr','gy_corr','gz_corr']].values
        mag = df[['mx','my','mz']].values
        gyro_rad = np.radians(gyro)
        acc_ms2 = acc * 9.80665

        # Step 3: Estimate Initial Orientation
        def estimate_initial_quaternion(acc_vec):
            g = acc_vec / np.linalg.norm(acc_vec)
            roll = np.arctan2(g[1], g[2])
            R0 = R.from_euler('Z', roll, degrees=False)
            return R0.as_quat()

        # Step 4: Initialize Madgwick Filter
        dt = np.mean(np.diff(df['time_s']))
        madgwick = Madgwick(sampleperiod=dt, beta=0.1)
        Q = np.zeros((len(df), 4))
        Q[0] = estimate_initial_quaternion(acc_ms2[0])

        # Step 5: Orientation Estimation
        for i in range(1, len(df)):
            Q[i] = madgwick.updateMARG(Q[i-1], gyr=gyro_rad[i], acc=acc_ms2[i], mag=mag[i])

        # Step 6: Convert to Euler Angles
        euler = R.from_quat(Q).as_euler('ZYX', degrees=True)
        euler = np.unwrap(np.radians(euler), axis=0)
        euler = np.degrees(euler)
        df['roll'], df['pitch'], df['yaw'] = euler[:, 0], euler[:, 1], euler[:, 2]

        # Step 7: Transform Acceleration to World Frame
        rotations = R.from_quat(Q)
        acc_world_raw = rotations.apply(acc_ms2)
        g_body = np.tile([0, 0, -9.80665], (len(df), 1))
        g_world_dynamic = rotations.apply(g_body)
        acc_world = acc_world_raw + g_world_dynamic

        # Step 8: Remove Postflight Residual Bias
        postflight = df[df['time_s'] > marker_data['landing']['time'] + 0.5]
        post_bias = np.mean(acc_world[-len(postflight):], axis=0)
        acc_world -= post_bias

        # Step 9: Save Transformed Acceleration
        df['ax_world'] = acc_world[:, 0]
        df['ay_world'] = acc_world[:, 1]
        df['az_world'] = acc_world[:, 2]

        # Step 10: Net Acceleration Comparison
        acc_body_no_grav  = acc_ms2 + rotations.inv().apply(np.array([0, 0, -9.80665]))
        acc_world_no_grav = acc_world_raw + g_world_dynamic
        df['acc_net_body']  = np.linalg.norm(acc_body_no_grav, axis=1)
        df['acc_net_world'] = np.linalg.norm(acc_world_no_grav, axis=1)
        df['acc_net_diff']  = df['acc_net_world'] - df['acc_net_body']

        # Step 11: Orientation Plot
        fig = go.Figure()
        for name, color in zip(['roll', 'pitch', 'yaw'], ['blue', 'orange', 'green']):
            fig.add_trace(go.Scatter(x=df['time_s'], y=df[name], name=name.capitalize(), line=dict(color=color)))

        mins = df[['roll','pitch','yaw']].min().min()
        maxs = df[['roll','pitch','yaw']].max().max()
        for ev in ['launch','burnout','apogee','parachute','landing']:
            te = marker_data[ev]['time']
            fig.add_shape(type='line', x0=te, x1=te, y0=mins, y1=maxs,
                          xref='x', yref='y', line=dict(color='gray', dash='dot'))
            fig.add_annotation(x=te, y=0, text=ev.capitalize(), showarrow=False, yshift=10)

        fig.update_layout(
            title="Orientation (Roll, Pitch, Yaw) Over Time - World Frame",
            xaxis_title="Time (s)", yaxis_title="Angle (°)",
            legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.05)
        )
        fig.show()

        # Step 12: Net Acceleration Plot
        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=df['time_s'], y=df['acc_net_body'],  name='Accel Body Frame', line=dict(color='blue')))
        fig2.add_trace(go.Scatter(x=df['time_s'], y=df['acc_net_world'], name='Accel World Frame', line=dict(color='green')))
        fig2.add_trace(go.Scatter(x=df['time_s'], y=df['acc_net_diff'],  name='Difference', line=dict(color='red', dash='dot')))
        fig2.update_layout(
            title="Acceleration: Body Frame vs. World Frame",
            xaxis_title="Time (s)", yaxis_title="Acceleration Magnitude (m/s²)",
            legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.01)
        )
        fig2.show()

        print("\n📊 Bias Summary:")
        print("Preflight accelerometer bias (g):", acc_bias)
        print("Postflight acceleration residual (world frame, m/s²):", post_bias)

run_button_8 = widgets.Button(description="▶ Run Step 8", button_style='primary')
run_output_8 = widgets.Output()
run_button_8.on_click(run_step_8)
container_8 = widgets.VBox([run_button_8, run_output_8])
display(container_8)

In [ ]:
# ===========================
# CELL 9 - Final Orientation Correction, Validation, and Visualization
# ===========================

from scipy.spatial.transform import Rotation as R
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Entry Point ---
run_button_9 = widgets.Button(description="▶ View Corrected World Orientation", button_style='primary')
output_9 = widgets.Output()

def run_step_9(b):
    with output_9:
        clear_output()

        angle_step = 2.0
        tolerance = 0.5

        euler_deg = df[['roll', 'pitch', 'yaw']].values
        initial_rotations = R.from_euler('xyz', euler_deg, degrees=True)

        acc_body = df[['ax_filtered', 'ay_filtered', 'az_filtered']].values
        gravity = np.array([0, 0, 9.807])
        corrected_rotations = []

        for i in range(len(df)):
            R_orig = initial_rotations[i]
            acc_w = R_orig.apply(acc_body[i])
            diff = np.linalg.norm(acc_w - gravity)

            best_rot = R_orig
            best_error = diff

            if diff > tolerance:
                for axis in ['x', 'y', 'z']:
                    for sign in [-1, 1]:
                        tweak = R.from_euler(axis, sign * angle_step, degrees=True)
                        R_new = tweak * R_orig
                        acc_w_tweaked = R_new.apply(acc_body[i])
                        new_error = np.linalg.norm(acc_w_tweaked - gravity)
                        if new_error < best_error:
                            best_rot = R_new
                            best_error = new_error

            corrected_rotations.append(best_rot)

        corrected_euler = R.from_quat([r.as_quat() for r in corrected_rotations]).as_euler('xyz', degrees=True)
        df['roll_corrected'], df['pitch_corrected'], df['yaw_corrected'] = corrected_euler.T

        corrected_rot = R.from_euler('xyz', corrected_euler, degrees=True)
        acc_world_corrected = corrected_rot.apply(acc_body)
        df['acc_world_corr_x'] = acc_world_corrected[:, 0]
        df['acc_world_corr_y'] = acc_world_corrected[:, 1]
        df['acc_world_corr_z'] = acc_world_corrected[:, 2]

        df['acc_mag_body'] = np.linalg.norm(acc_body, axis=1)
        df['acc_mag_world_corr'] = np.linalg.norm(acc_world_corrected, axis=1)
        df['acc_mag_diff_corr'] = df['acc_mag_body'] - df['acc_mag_world_corr']

        roll_unwrapped = np.unwrap(np.radians(df['roll_corrected'].values))
        pitch_unwrapped = np.unwrap(np.radians(df['pitch_corrected'].values))
        yaw_unwrapped = np.unwrap(np.radians(df['yaw_corrected'].values))
        roll_unwrapped_deg = np.degrees(roll_unwrapped)
        pitch_unwrapped_deg = np.degrees(pitch_unwrapped)
        yaw_unwrapped_deg = np.degrees(yaw_unwrapped)

        t = df['time_s'].values
        launch_t = marker_data['launch']['time']
        landing_t = marker_data['landing']['time']

        def flatten_edges(angle_data, t, launch_t, landing_t):
            flat_data = angle_data.copy()
            mask_pre = t < launch_t
            if mask_pre.any():
                model = LinearRegression()
                model.fit(t[mask_pre].reshape(-1, 1), angle_data[mask_pre])
                correction = model.predict(t[mask_pre].reshape(-1, 1))
                flat_data[mask_pre] -= (correction - correction[-1])
            mask_post = t > landing_t
            if mask_post.any():
                model = LinearRegression()
                model.fit(t[mask_post].reshape(-1, 1), angle_data[mask_post])
                correction = model.predict(t[mask_post].reshape(-1, 1))
                flat_data[mask_post] -= (correction - correction[0])
            return flat_data

        df['roll_flat'] = flatten_edges(df['roll_corrected'].values.copy(), t, launch_t, landing_t)
        df['pitch_flat'] = flatten_edges(df['pitch_corrected'].values.copy(), t, launch_t, landing_t)
        df['yaw_flat'] = flatten_edges(df['yaw_corrected'].values.copy(), t, launch_t, landing_t)

        plt.figure(figsize=(12, 6))
        plt.plot(df['time_s'], df['acc_mag_body'], label='Body Accel Magnitude (Filtered)', alpha=0.8)
        plt.plot(df['time_s'], df['acc_mag_world_corr'], label='World Accel Magnitude (Corrected)', alpha=0.8)
        plt.plot(df['time_s'], df['acc_mag_diff_corr'], label='Magnitude Difference', linestyle='--', alpha=0.6)
        plt.xlabel('Time (s)')
        plt.ylabel('Acceleration (m/s²)')
        plt.title('Body vs World Frame Acceleration Magnitudes (Corrected)')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        roll_flat_unwrapped = np.unwrap(np.radians(df['roll_flat'].values))
        pitch_flat_unwrapped = np.unwrap(np.radians(df['pitch_flat'].values))
        yaw_flat_unwrapped = np.unwrap(np.radians(df['yaw_flat'].values))
        roll_flat_unwrapped_deg = np.degrees(roll_flat_unwrapped)
        pitch_flat_unwrapped_deg = np.degrees(pitch_flat_unwrapped)
        yaw_flat_unwrapped_deg = np.degrees(yaw_flat_unwrapped)

        plt.figure(figsize=(12, 6))
        plt.plot(df['time_s'], roll_unwrapped_deg, '--', label='Roll (Unwrapped)', color='red')
        plt.plot(df['time_s'], roll_flat_unwrapped_deg, '-', label='Roll (Flattened)', color='red')
        plt.plot(df['time_s'], pitch_unwrapped_deg, '--', label='Pitch (Unwrapped)', color='green')
        plt.plot(df['time_s'], pitch_flat_unwrapped_deg, '-', label='Pitch (Flattened)', color='green')
        plt.plot(df['time_s'], yaw_unwrapped_deg, '--', label='Yaw (Unwrapped)', color='blue')
        plt.plot(df['time_s'], yaw_flat_unwrapped_deg, '-', label='Yaw (Flattened)', color='blue')
        plt.xlabel("Time (s)")
        plt.ylabel("Degrees")
        plt.title("Unwrapped vs. Flattened Orientation (Corrected)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        print("✅ Final orientation correction, magnitude validation, and drift flattening complete.")

run_button_9.on_click(run_step_9)
display(run_button_9, output_9)

In [ ]:
# ===========================
# FIXED CELL 10 – Safe Kalman Trigger for Voilà
# ===========================

import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from filterpy.kalman import KalmanFilter

accel_conf_slider = widgets.FloatLogSlider(
    value=1.0, base=10, min=-2, max=2, step=0.1,
    description='Accel Confidence (R)',
    style={'description_width': '160px'},
    layout=widgets.Layout(width='50%')
)
run_button_10 = widgets.Button(description="▶ Run Z-Axis Kalman Filter", button_style='success')
output_10 = widgets.Output()

def run_kalman_filter(R_val):
    if not all(col in df.columns for col in ['time_s', 'altitude_m', 'vel_mps', 'acc_world_corr_z']):
        print("⚠ Required columns not ready.")
        return

    dt_vals = df['time_s'].diff().fillna(0).values
    n = len(df)
    alt_meas = df['altitude_m'].values
    vel_meas = df['vel_mps'].values
    acc_input = df['acc_world_corr_z'].fillna(0).values
    Z = np.vstack((alt_meas, vel_meas)).T

    kf = KalmanFilter(dim_x=2, dim_z=2)
    kf.x = np.array([[alt_meas[0]], [vel_meas[0]]])
    kf.H = np.eye(2)
    kf.P *= 500.
    kf.R = np.eye(2) * R_val

    z_out, vz_out = [], []

    for i in range(n):
        dt = dt_vals[i]
        kf.F = np.array([[1, dt], [0, 1]])
        kf.B = np.array([[0.5 * dt ** 2], [dt]])
        kf.Q = np.array([[0.25 * dt**4, 0.5 * dt**3],
                         [0.5 * dt**3, dt**2]]) * 10
        kf.predict(u=acc_input[i])
        kf.update(Z[i])
        z_out.append(kf.x[0, 0])
        vz_out.append(kf.x[1, 0])

    df['z_kalman_combo'] = z_out
    df['vz_kalman_combo'] = vz_out

def run_step_10(b=None):
    with output_10:
        clear_output(wait=True)
        R_val = accel_conf_slider.value
        run_kalman_filter(R_val)

        if 'z_kalman_combo' not in df.columns:
            print("⚠ Kalman filter output missing. Check earlier steps.")
            return

        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(x=df['time_s'], y=df['altitude_m'], name='Hypsometric Altitude', line=dict(color='green')))
        fig1.add_trace(go.Scatter(x=df['time_s'], y=df['z_kalman_combo'], name='Kalman Altitude', line=dict(color='black', dash='dash')))
        fig1.update_layout(title=f"Altitude Comparison — R={R_val:.2f}",
                           xaxis_title="Time (s)", yaxis_title="Altitude (m)",
                           height=400, width=800)

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=df['time_s'], y=df['vel_mps'], name='Hypsometric Velocity', line=dict(color='blue')))
        fig2.add_trace(go.Scatter(x=df['time_s'], y=df['vz_kalman_combo'], name='Kalman Velocity', line=dict(color='black', dash='dot')))
        fig2.update_layout(title=f"Velocity Comparison — R={R_val:.2f}",
                           xaxis_title="Time (s)", yaxis_title="Velocity (m/s)",
                           height=400, width=800)

        display(fig1, fig2)

run_button_10.on_click(run_step_10)
display(widgets.VBox([widgets.HBox([accel_conf_slider, run_button_10]), output_10]))

In [ ]:
# ===========================
# FINALIZED CELL 11 – Voilà-Safe 3D Path Editor
# ===========================

def run_step_11():
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    global event_coords  # <-- Expose globally for later cells

    output = widgets.Output()
    path_output = widgets.Output()

    event_coords = {
        'Launch':    [0, 0, 0],
        'Burnout':   [0, 0, 0],
        'Apogee':    [0, 0, 0],
        'Parachute': [0, 0, 0],
        'Landing':   [0, 0, 0]
    }

    dropdown = widgets.Dropdown(options=[k for k in event_coords if k != 'Launch'], description='Event:')
    x_slider = widgets.FloatSlider(description='Y (East)', min=-300, max=300, step=1)
    y_slider = widgets.FloatSlider(description='X (North)', min=-300, max=300, step=1)
    build_button = widgets.Button(description="Generate Path", button_style='success')
    load_button = widgets.Button(description="▶ Estimate Event Positions", button_style='primary')

    def draw_plot():
        with output:
            clear_output(wait=True)
            fig = go.Figure()
            pts = [np.array(event_coords[k]) for k in ['Launch','Burnout','Apogee','Parachute','Landing']]
            xs, ys, zs = [pts[0][0]], [pts[0][1]], [pts[0][2]]
            t_vals = np.linspace(0, 2, 100)

            def parabola(p0, p1, p2):
                A = np.array([[0**2, 0, 1], [1**2, 1, 1], [2**2, 2, 1]])
                return [np.linalg.solve(A, [p0[i], p1[i], p2[i]]) for i in range(3)]

            cx, cy, cz = parabola(*pts[1:4])
            xs += list(cx[0]*t_vals**2 + cx[1]*t_vals + cx[2])
            ys += list(cy[0]*t_vals**2 + cy[1]*t_vals + cy[2])
            zs += list(cz[0]*t_vals**2 + cz[1]*t_vals + cz[2])
            xs += [pts[-1][0]]
            ys += [pts[-1][1]]
            zs += [pts[-1][2]]

            fig.add_trace(go.Scatter3d(x=xs, y=ys, z=zs, mode='lines', line=dict(color='blue', width=4)))
            for k, (x, y, z) in event_coords.items():
                fig.add_trace(go.Scatter3d(x=[x], y=[y], z=[z], mode='markers+text',
                                           marker=dict(size=6), text=[k], textposition='top center'))

            scene_camera = dict(eye=dict(x=-3, y=-3, z=2), center=dict(x=0, y=0, z=0), up=dict(x=0, y=0, z=1))

            fig.update_layout(
                scene=dict(
                    xaxis_title='Y (East, m)',
                    yaxis_title='X (North, m)',
                    zaxis_title='Altitude (m)',
                    aspectmode='data'
                ),
                scene_camera=scene_camera,
                title="Rocket Flight Path Editor",
                margin=dict(l=0, r=0, b=0, t=30)
            )
            fig.show()

    def update_sliders(change=None):
        e = dropdown.value
        x_slider.value = event_coords[e][0]
        y_slider.value = event_coords[e][1]

    def update_coords(change=None):
        e = dropdown.value
        event_coords[e][0] = x_slider.value
        event_coords[e][1] = y_slider.value
        draw_plot()

    def interpolate_segment(p0, p1, t0, t1, kind='linear'):
        n = int((t1 - t0) * 100)
        t_vals = np.linspace(t0, t1, n)
        x = np.linspace(p0[0], p1[0], n)
        y = np.linspace(p0[1], p1[1], n)
        if kind == 'linear':
            z = np.linspace(p0[2], p1[2], n)
        elif kind == 'parabolic':
            h = (t0 + t1) / 2
            a = -4 * (p1[2] - p0[2]) / ((t1 - t0) ** 2)
            z = a * (t_vals - h)**2 + p1[2]
        return pd.DataFrame({'time_s': t_vals, 'x_m': x, 'y_m': y, 'z_m': z})

    def build_path(_):
        global df_flight_path
        with path_output:
            path_output.clear_output()
            t = {k: marker_data[k]['time'] for k in ['launch','burnout','apogee','parachute','landing']}
            xyz = {k: np.array(event_coords[k.capitalize()]) for k in t}
            df_flight_path = pd.concat([
                interpolate_segment([0, 0, 0], xyz['burnout'], t['launch'], t['burnout']),
                interpolate_segment(xyz['burnout'], xyz['apogee'], t['burnout'], t['apogee'], 'parabolic'),
                interpolate_segment(xyz['apogee'], xyz['parachute'], t['apogee'], t['parachute'], 'parabolic'),
                interpolate_segment(xyz['parachute'], xyz['landing'], t['parachute'], t['landing'])
            ], ignore_index=True)
            print("✅ Flight path saved to `df_flight_path`.")

    def load_data(_):
        az1 = displacement_data['azimuth_launch_to_apogee']
        dx = displacement_data['dx']
        dy = displacement_data['dy']
        apogee_dist = displacement_data['distance'] * 0.5
        az1_rad = np.radians(az1)
        apogee_x = apogee_dist * np.cos(az1_rad)
        apogee_y = apogee_dist * np.sin(az1_rad)

        def get_altitude(t): return np.interp(t, df['time_s'], df['altitude_m'])

        event_coords.update({
            'Launch':    [0, 0, get_altitude(marker_data['launch']['time'])],
            'Burnout':   [apogee_y * 0.4, apogee_x * 0.4, get_altitude(marker_data['burnout']['time'])],
            'Apogee':    [apogee_y, apogee_x, get_altitude(marker_data['apogee']['time'])],
            'Parachute': [dy * 0.6, dx * 0.6, get_altitude(marker_data['parachute']['time'])],
            'Landing':   [dy, dx, get_altitude(marker_data['landing']['time'])]
        })
        update_sliders()
        draw_plot()

    x_slider.observe(update_coords, names='value')
    y_slider.observe(update_coords, names='value')
    dropdown.observe(update_sliders, names='value')
    build_button.on_click(build_path)
    load_button.on_click(load_data)

    update_sliders()
    draw_plot()

    display(widgets.VBox([
        load_button,
        widgets.HTML("<h4>🎯 Adjust Event Positions</h4>"),
        dropdown, x_slider, y_slider,
        output, build_button, path_output
    ]))

# Trigger the function to define everything
run_step_11()


In [ ]:
# ===========================
# CELL 12 – 3D Position Components vs. Time (with Flat Extensions and Hypsometric Altitude)
# ===========================

import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

output_12 = widgets.Output()

# --- Run function ---
def run_step_12(b=None):
    with output_12:
        clear_output(wait=True)

        # --- Extract key event times ---
        t_launch = marker_data['launch']['time']
        t_burnout = marker_data['burnout']['time']
        t_apogee = marker_data['apogee']['time']
        t_parachute = marker_data['parachute']['time']
        t_landing = marker_data['landing']['time']

        # --- Extract 3D positions at each event (X = East, Y = North, Z = Altitude) ---
        x_vals = [event_coords['Launch'][0], event_coords['Burnout'][0],
                  event_coords['Apogee'][0], event_coords['Parachute'][0], event_coords['Landing'][0]]
        y_vals = [event_coords['Launch'][1], event_coords['Burnout'][1],
                  event_coords['Apogee'][1], event_coords['Parachute'][1], event_coords['Landing'][1]]
        z_vals = [event_coords['Launch'][2], event_coords['Burnout'][2],
                  event_coords['Apogee'][2], event_coords['Parachute'][2], event_coords['Landing'][2]]

        # --- Time values in matching order ---
        time_vals = [t_launch, t_burnout, t_apogee, t_parachute, t_landing]

        # --- Add flat extensions pre/post flight to anchor position at launch and landing ---
        t_pre = df['time_s'].min()
        t_post = df['time_s'].max()

        time_extended = [t_pre] + time_vals + [t_post]
        x_extended = [x_vals[0]] + x_vals + [x_vals[-1]]
        y_extended = [y_vals[0]] + y_vals + [y_vals[-1]]
        z_extended = [z_vals[0]] + z_vals + [z_vals[-1]]

        # --- Plot 3D position components over time ---
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=time_extended, y=x_extended, mode='lines+markers',
                                 name='X (East)', line=dict(color='red')))
        fig.add_trace(go.Scatter(x=time_extended, y=y_extended, mode='lines+markers',
                                 name='Y (North)', line=dict(color='green')))
        fig.add_trace(go.Scatter(x=time_extended, y=z_extended, mode='lines+markers',
                                 name='Z (Altitude)', line=dict(color='blue')))

        # --- Add Hypsometric Altitude for reference ---
        fig.add_trace(go.Scatter(x=df['time_s'], y=df['altitude_m'],
                                 mode='lines', name='Hypsometric Altitude',
                                 line=dict(color='black', dash='dot')))

        # --- Layout settings ---
        fig.update_layout(
            title="3D Position Components vs Time (with Hypsometric Altitude)",
            xaxis_title="Time (s)",
            yaxis_title="Position (meters)",
            height=500,
            width=900,
            legend=dict(x=0, y=1),
            margin=dict(l=20, r=20, t=40, b=20)
        )

        fig.show()

# --- Button setup ---
button12 = widgets.Button(description="▶ View Position Estimations", button_style='primary')
button12.on_click(run_step_12)

display(widgets.VBox([button12, output_12]))


In [ ]:
# ===========================
# CELL 13 - Kalman Filter for X and Y Position and Velocity
# ===========================

import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.interpolate import interp1d
from filterpy.kalman import KalmanFilter

# --- Widgets ---
r_slider = widgets.FloatLogSlider(
    value=10.0,
    base=10,
    min=-2,
    max=2,
    step=0.1,
    description='R (Meas Noise)',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='50%')
)
run_button_13 = widgets.Button(description="▶ Run X&Y Kalman Filter", button_style='primary')
output = widgets.Output()

# --- Kalman Filter Function ---
def run_kalman_xy_filter(R_val):
    time_vals = df['time_s'].values
    dt_vals = df['time_s'].diff().fillna(0).values
    n = len(df)

    # Define event times within this scope
    t_launch = marker_data['launch']['time']
    t_burnout = marker_data['burnout']['time']
    t_apogee = marker_data['apogee']['time']
    t_parachute = marker_data['parachute']['time']
    t_landing = marker_data['landing']['time']

    extended_times = np.array([time_vals[0], t_launch, t_burnout, t_apogee, t_parachute, t_landing, time_vals[-1]])
    x_events = np.array([event_coords['Launch'][0], event_coords['Launch'][0], event_coords['Burnout'][0],
                         event_coords['Apogee'][0], event_coords['Parachute'][0],
                         event_coords['Landing'][0], event_coords['Landing'][0]])
    y_events = np.array([event_coords['Launch'][1], event_coords['Launch'][1], event_coords['Burnout'][1],
                         event_coords['Apogee'][1], event_coords['Parachute'][1],
                         event_coords['Landing'][1], event_coords['Landing'][1]])

    interp_x = interp1d(extended_times, x_events, kind='linear', bounds_error=False, fill_value="extrapolate")
    interp_y = interp1d(extended_times, y_events, kind='linear', bounds_error=False, fill_value="extrapolate")

    x_meas = interp_x(time_vals)
    y_meas = interp_y(time_vals)

    def kalman_axis(axis_label, acc_data, meas_data):
        kf = KalmanFilter(dim_x=2, dim_z=1)
        kf.x = np.array([[meas_data[0]], [0]])
        kf.H = np.array([[1, 0]])
        kf.P *= 500.
        kf.R *= R_val

        pos_out, vel_out = [], []

        for i in range(n):
            dt = dt_vals[i]
            kf.F = np.array([[1, dt], [0, 1]])
            kf.B = np.array([[0.5 * dt**2], [dt]])
            kf.Q = np.array([[0.25 * dt**4, 0.5 * dt**3],
                             [0.5 * dt**3, dt**2]]) * 5.0

            kf.predict(u=acc_data[i])
            kf.update(np.array([[meas_data[i]]]))

            if time_vals[i] < t_launch or time_vals[i] > t_landing:
                kf.x[1, 0] = 0.0

            pos_out.append(kf.x[0, 0])
            vel_out.append(kf.x[1, 0])

        df[f'{axis_label}_kalman'] = pos_out
        df[f'v{axis_label}_kalman'] = vel_out

    kalman_axis('x', df['acc_world_corr_x'].fillna(0).values, x_meas)
    kalman_axis('y', df['acc_world_corr_y'].fillna(0).values, y_meas)

    return time_vals, x_meas, y_meas

# --- Callback for plotting ---
def run_step_13(b=None):
    with output:
        clear_output(wait=True)
        R_val = r_slider.value
        t, x_meas, y_meas = run_kalman_xy_filter(R_val)

        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(x=t, y=x_meas, mode='lines', name='X Interp', line=dict(color='red')))
        fig1.add_trace(go.Scatter(x=t, y=df['x_kalman'], mode='lines', name='X Kalman', line=dict(color='black', dash='dot')))
        fig1.add_trace(go.Scatter(x=t, y=y_meas, mode='lines', name='Y Interp', line=dict(color='green')))
        fig1.add_trace(go.Scatter(x=t, y=df['y_kalman'], mode='lines', name='Y Kalman', line=dict(color='black', dash='dash')))
        fig1.update_layout(title=f"X and Y Position — R={R_val:.2f}",
                           xaxis_title="Time (s)", yaxis_title="Position (m)",
                           height=400, width=900)

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=t, y=df['vx_kalman'], mode='lines', name='Vx Kalman', line=dict(color='red')))
        fig2.add_trace(go.Scatter(x=t, y=df['vy_kalman'], mode='lines', name='Vy Kalman', line=dict(color='green')))
        fig2.update_layout(title=f"X and Y Velocity — R={R_val:.2f}",
                           xaxis_title="Time (s)", yaxis_title="Velocity (m/s)",
                           height=400, width=900)

        fig3 = go.Figure()
        fig3.add_trace(go.Scatter3d(x=df['x_kalman'], y=df['y_kalman'], z=df['z_kalman_combo'],
                                    mode='lines', line=dict(color='blue', width=4), name='Kalman Trajectory'))

        for label in ['Launch', 'Burnout', 'Apogee', 'Parachute', 'Landing']:
            t_event = marker_data[label.lower()]['time']
            idx = (np.abs(df['time_s'] - t_event)).idxmin()
            fig3.add_trace(go.Scatter3d(
                x=[df.loc[idx, 'x_kalman']], y=[df.loc[idx, 'y_kalman']], z=[df.loc[idx, 'z_kalman_combo']],
                mode='markers+text', marker=dict(size=6),
                text=[label], textposition='top center', name=label
            ))

        fig3.update_layout(scene=dict(
            xaxis_title='Y (East, m)',
            yaxis_title='X (North, m)',
            zaxis_title='Altitude (m)',
            aspectmode='data',
            camera=dict(eye=dict(x=-1.5, y=-1.5, z=1), up=dict(x=0, y=0, z=1))
        ), title='3D Kalman Filtered Rocket Trajectory with Event Markers',
           margin=dict(l=0, r=0, b=0, t=30), height=600, width=900)

        display(fig1, fig2, fig3)

run_button_13.on_click(run_step_13)
display(widgets.VBox([widgets.HBox([r_slider, run_button_13]), output]))

In [2]:
# ===========================
# FINALIZED CELL 14 – Working Animation with Locked Camera
# ===========================

import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio

# --- UI Container ---
run_button_14 = widgets.Button(description="▶ Run Flight Simulation", button_style='primary')
ui_output = widgets.Output()

# --- Event Handler ---
def run_step_14(b):
    with ui_output:
        clear_output()

        t_launch = marker_data['launch']['time']
        t_landing = marker_data['landing']['time']
        t_min = max(df['time_s'].min(), t_launch - 5)
        t_max = min(df['time_s'].max(), t_landing + 5)

        def format_time_label(t):
            delta = t - t_launch
            sign = "+" if delta >= 0 else "-"
            mm, ss = divmod(abs(delta), 60)
            return f"T{sign}{int(mm):02d}:{ss:04.1f}"

        # Precompute arrays
        time_vals = df['time_s'].values
        x_vals = df['x_kalman'].values
        y_vals = df['y_kalman'].values
        z_vals = df['z_kalman_combo'].values

        # Create figure and traces
        fig = go.FigureWidget()
        fig.add_trace(go.Scatter3d(mode='lines', line=dict(color='blue', width=4), name='Trajectory'))
        fig.add_trace(go.Scatter3d(mode='markers', marker=dict(size=6, color='red'), name='Rocket'))

        # Event markers
        for label in ['Launch', 'Burnout', 'Apogee', 'Parachute', 'Landing']:
            t_event = marker_data[label.lower()]['time']
            idx = (abs(df['time_s'] - t_event)).idxmin()
            fig.add_trace(go.Scatter3d(
                x=[df.loc[idx, 'x_kalman']],
                y=[df.loc[idx, 'y_kalman']],
                z=[df.loc[idx, 'z_kalman_combo']],
                mode='markers+text',
                marker=dict(size=5),
                text=[label],
                textposition='top center',
                name=label
            ))

        fig.update_layout(
            scene=dict(
                xaxis_title='Y (East, m)', yaxis_title='X (North, m)', zaxis_title='Altitude (m)',
                aspectmode='data', camera=dict(eye=dict(x=-1.5, y=-1.5, z=1))
            ),
            title="Kalman Trajectory",
            margin=dict(l=0, r=0, b=0, t=30),
            height=600
        )

        time_slider = widgets.FloatSlider(
            value=t_min, min=t_min, max=t_max, step=0.1,
            description='Time', continuous_update=True,
            layout=widgets.Layout(width='75%')
        )
        play_button = widgets.ToggleButton(value=False, description="Play", icon='play', button_style='success')
        label_output = widgets.Output()

        def update_plot(t_now):
            mask = (time_vals >= t_min) & (time_vals <= t_now)
            x_traj = x_vals[mask]
            y_traj = y_vals[mask]
            z_traj = z_vals[mask]

            fig.data[0].x = x_traj
            fig.data[0].y = y_traj
            fig.data[0].z = z_traj

            if len(x_traj) > 0:
                fig.data[1].x = [x_traj[-1]]
                fig.data[1].y = [y_traj[-1]]
                fig.data[1].z = [z_traj[-1]]
            else:
                fig.data[1].x = []
                fig.data[1].y = []
                fig.data[1].z = []

            with label_output:
                clear_output(wait=True)
                print(f"Time: {format_time_label(t_now)}")

        async def play_animation():
            play_button.icon = 'pause'
            play_button.description = 'Pause'
            while play_button.value and time_slider.value < t_max:
                time_slider.value = round(time_slider.value + 0.1, 1)
                await asyncio.sleep(0.05)
            play_button.value = False
            play_button.icon = 'play'
            play_button.description = 'Play'

        def on_slider_change(change):
            update_plot(change.new)

        def on_play_toggle(change):
            if play_button.value:
                asyncio.ensure_future(play_animation())

        time_slider.observe(on_slider_change, names='value')
        play_button.observe(on_play_toggle, names='value')

        display(widgets.VBox([
            widgets.HBox([time_slider, play_button]),
            label_output,
            fig
        ]))
        update_plot(t_min)

display(widgets.VBox([run_button_14, ui_output]))
run_button_14.on_click(run_step_14)